# CIFAR-10 Image Classification Using Convolutional Neural Networks
**Author:** Virginia Herrero

## Introduction

This project explores how to use a Convolutional Neural Network (CNN) to classify images from the CIFAR-10 dataset. CNNs are a type of deep learning model that work especially well with image data because they can automatically learn to detect patterns like edges, shapes, and textures. 

## Goal

The main goal is to build and train a CNN that can accurately classify images from the CIFAR-10 dataset. Along the way, this involves a few key steps: preparing the data, designing the model architecture, training it on the dataset, and evaluating how well it performs. There's also room to experiment with improvements like data augmentation or tuning the model to boost accuracy.

## Dataset Overview

CIFAR-10 is a popular dataset used for training machine learning models on image classification tasks. It contains **60,000 color images**, each just **32x32 pixels**, across **10 different categories**. There are **50,000 images for training** and **10,000 for testing**.

The ten classes are:

* Airplane
* Automobile
* Bird
* Cat
* Deer
* Dog
* Frog
* Horse
* Ship
* Truck

Each image is labeled with one of these classes. While the images are small and simple, they still represent a good challenge for building a working image classifier.